In [ ]:
import pandas as pd

# Input Excel file and sheet name
input_excel = "question.xlsx"  # Replace with your Excel file name
sheet_name = "Eng"  # Name of the sheet to read from
query_column = "Mysql correct query"  # Column name containing the SQL queries

# Output: List of SQL queries
queries = []

def read_queries_from_excel(excel_file, sheet_name, query_column):
    try:
        # Read the Excel file
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Extract the 'Mysql correct query' column
        if query_column in df.columns:
            queries.extend(df[query_column].dropna().tolist())  # Drop any NaN values and convert to list
        else:
            print(f"Column '{query_column}' not found in the sheet.")
        print("Queries loaded successfully.")
        return queries
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Read the queries from the Excel file
queries = read_queries_from_excel(input_excel, sheet_name, query_column)

# Print the generated queries
for i, query in enumerate(queries, start=1):
    print(f"Query {i}: {query}")


Queries loaded successfully.
Query 1: SELECT total_assets FROM financialmetrics WHERE company_id = (SELECT id FROM company WHERE symbol = 'ADVANC') AND period_id = (SELECT id FROM period WHERE year = 2019 AND quarter = 1);
Query 2: SELECT total_liabilities FROM financialmetrics WHERE company_id = (SELECT id FROM company WHERE symbol = 'AOT') AND period_id = (SELECT id FROM period WHERE year = 2019 AND quarter = 1);
Query 3: SELECT net_profit_quarter FROM financialmetrics WHERE company_id = (SELECT id FROM company WHERE symbol = 'BBL') AND period_id = (SELECT id FROM period WHERE year = 2019 AND quarter = 1);
Query 4: SELECT value FROM financialratios WHERE company_id = (SELECT id FROM company WHERE symbol = 'BCP') AND period_id = (SELECT id FROM period WHERE year = 2019 AND quarter = 1) AND type = 'ROE';
Query 5: SELECT value FROM financialratios WHERE company_id = (SELECT id FROM company WHERE symbol = 'BDMS') AND period_id = (SELECT id FROM period WHERE year = 2019 AND quarter = 1) A

In [1]:
queries

NameError: name 'queries' is not defined

In [ ]:

len(queries)

100

In [ ]:
import time
import psutil
import os
import json
import mysql.connector
import csv
from decimal import Decimal
from datetime import date, datetime

# MySQL connection setup
connection = mysql.connector.connect(
    host="localhost",
    user="root",  # Replace with your MySQL username
    password="Wealth3visual%",  # Replace with your MySQL password
    database="set50"  # Replace with your database name
)
cursor = connection.cursor(dictionary=True)

# Output file
output_csv = "query_results_with_metrics_mysql.csv"
repetitions = 10  # Number of times to execute each query for averaging

# Function to monitor RAM usage
def get_ram_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB

# Function to convert result to JSON-serializable format
def convert_to_serializable(data):
    if isinstance(data, dict):
        return {k: (float(v) if isinstance(v, Decimal) else str(v) if isinstance(v, (date, datetime)) else v) 
                for k, v in data.items()}
    # Handle cases where the result is a list or other data structure
    elif isinstance(data, (list, tuple)):
        return [convert_to_serializable(item) for item in data]
    return data

# Function to measure query execution time with averaging
def measure_query_time(query, repetitions):
    total_time = 0
    for _ in range(repetitions):
        start_time = time.time()
        cursor.execute(query)
        cursor.fetchall()  # Fetch all results to ensure the query completes
        end_time = time.time()
        total_time += (end_time - start_time)
    return total_time / repetitions

# Function to run queries and save results
def run_queries_and_export_with_metrics(queries, output_csv):
    try:
        # List to store results
        all_results = []

        for index, query in enumerate(queries):
            # Measure RAM before execution
            start_ram = get_ram_usage()

            # Measure execution time with averaging
            avg_execution_time = measure_query_time(query, repetitions)

            # Get the actual query result (run once)
            cursor.execute(query)
            result = cursor.fetchone()

            # Ensure we consume any leftover results
            cursor.fetchall()  # This ensures that the cursor is fully consumed

            # Measure RAM after execution
            end_ram = get_ram_usage()

            # Calculate RAM usage
            ram_usage = end_ram - start_ram

            # Prepare row data
            row = {
                "query_index": index + 1,
                "query": query,
                "result": json.dumps(convert_to_serializable(result)) if result else "NULL",
                "time": avg_execution_time,  # Average time in seconds
                "ram": ram_usage  # RAM usage in MB
            }
            all_results.append(row)

        # Write results to CSV
        if all_results:
            with open(output_csv, mode="w", encoding="utf-8-sig", newline="") as file:
                writer = csv.DictWriter(file, fieldnames=all_results[0].keys())
                writer.writeheader()
                writer.writerows(all_results)

        print(f"Results with metrics exported to {output_csv}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        connection.close()


# Execute the function
run_queries_and_export_with_metrics(queries, output_csv)


Results with metrics exported to query_results_with_metrics_mysql.csv
